# Importe und Konfiguration
Importiere pandas, numpy, seaborn, matplotlib, sqlalchemy, psycopg2. Setze Plot-Styles, globale Konstanten (Zeitraum 2013-01-01 bis 2025-12-31) und lade DB-DSN aus Umgebungsvariable oder Eingabe.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import os

# Set plot styles
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Define global constants
START_DATE = "2013-01-01"
END_DATE = "2025-12-31"

# Load database DSN from environment variable or user input
DB_DSN = os.getenv("DB_DSN") or input("Bitte geben Sie den Datenbank-DSN ein: ")

# Create a database engine
engine = create_engine(DB_DSN)

# Datenbankverbindung (SQLAlchemy) testen
Erzeuge eine SQLAlchemy Engine (postgresql+psycopg2) und teste eine einfache SELECT 1-Abfrage. Definiere Hilfsfunktion sql_to_df(query, params) für DataFrame-Imports.

In [ ]:
# Test the database connection and define a helper function for SQL queries

# Test the database connection with a simple SELECT 1 query
with engine.connect() as connection:
    result = connection.execute("SELECT 1")
    assert result.scalar() == 1, "Database connection test failed."

# Define a helper function to execute SQL queries and return a DataFrame
def sql_to_df(query, params=None):
    """
    Execute a SQL query and return the result as a pandas DataFrame.

    Parameters:
        query (str): The SQL query to execute.
        params (dict, optional): Parameters to pass to the query.

    Returns:
        pd.DataFrame: The query result as a DataFrame.
    """
    with engine.connect() as connection:
        return pd.read_sql_query(query, connection, params=params)

# Query: Zeitreihe-Dataset (Median-Tag-Datum je Repo, Repos mit ≥1 Secret, Monats-Binning 2013–2025)
CTEs: (1) tag_dates: SELECT repos.id AS repo_id, COALESCE(tags.last_pushed, tags.last_pulled) AS dt JOIN tags→repos; (2) repo_median AS SELECT repo_id, percentile_cont(0.5) WITHIN GROUP (ORDER BY dt) AS med_dt FROM tag_dates GROUP BY repo_id; (3) repo_has_secret: DISTINCT repo_id via repo_layers JOIN layer_secret_fragments; (4) monthly_counts: SELECT date_trunc('month', med_dt) AS mon, COUNT(DISTINCT repo_id) FROM repo_median JOIN repo_has_secret USING(repo_id) WHERE med_dt BETWEEN :start AND :end GROUP BY 1 ORDER BY 1. Lade in DataFrame und fülle fehlende Monate mit 0.

In [ ]:
# Define the SQL query to generate the Zeitreihe dataset
query = """
WITH tag_dates AS (
    SELECT 
        repos.id AS repo_id, 
        COALESCE(tags.last_pushed, tags.last_pulled) AS dt
    FROM tags
    JOIN repos ON tags.repo_name = repos.repo_name
),
repo_median AS (
    SELECT 
        repo_id, 
        percentile_cont(0.5) WITHIN GROUP (ORDER BY dt) AS med_dt
    FROM tag_dates
    GROUP BY repo_id
),
repo_has_secret AS (
    SELECT DISTINCT 
        repo_id
    FROM repo_layers
    JOIN layer_secret_fragments ON repo_layers.layer_id = layer_secret_fragments.layer_id
),
monthly_counts AS (
    SELECT 
        date_trunc('month', med_dt) AS mon, 
        COUNT(DISTINCT repo_id) AS repo_count
    FROM repo_median
    JOIN repo_has_secret USING (repo_id)
    WHERE med_dt BETWEEN :start AND :end
    GROUP BY 1
    ORDER BY 1
)
SELECT 
    mon, 
    COALESCE(repo_count, 0) AS repo_count
FROM generate_series(:start::date, :end::date, '1 month') AS mon
LEFT JOIN monthly_counts ON mon.mon = monthly_counts.mon
ORDER BY mon;
"""

# Execute the query and load the result into a DataFrame
zeitreihe_df = sql_to_df(query, params={"start": START_DATE, "end": END_DATE})

# Fill missing months with 0 (if not already handled in the query)
zeitreihe_df["repo_count"] = zeitreihe_df["repo_count"].fillna(0)

# Display the first few rows of the DataFrame
zeitreihe_df.head()

# Plot: Zeitgraph Repos mit ≥1 Secret pro Monat
Linienplot mit Datum auf x, Anzahl Repos auf y; Achsenbeschriftungen, Limits (2013–2025), Gitter, Rotation der Tick-Labels.

In [ ]:
# Plot the time series graph for repositories with ≥1 secret per month
plt.figure(figsize=(12, 6))
plt.plot(zeitreihe_df["mon"], zeitreihe_df["repo_count"], marker="o", linestyle="-", color="b", label="Repos with ≥1 Secret")

# Set axis labels and title
plt.xlabel("Date", fontsize=12)
plt.ylabel("Number of Repositories", fontsize=12)
plt.title("Repositories with ≥1 Secret per Month (2013–2025)", fontsize=14)

# Set x-axis limits and rotate tick labels
plt.xlim(pd.Timestamp(START_DATE), pd.Timestamp(END_DATE))
plt.xticks(rotation=45)

# Add grid and legend
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

# Query: Top-10 Secret-Arten (origin)
SELECT origin, COUNT(*) AS c FROM secret_fragments WHERE origin IS NOT NULL AND origin <> '' GROUP BY origin ORDER BY c DESC LIMIT 10. In DataFrame laden.

In [ ]:
# Define the SQL query to get the top 10 secret origins
query_top_origins = """
SELECT 
    origin, 
    COUNT(*) AS c 
FROM secret_fragments 
WHERE origin IS NOT NULL AND origin <> '' 
GROUP BY origin 
ORDER BY c DESC 
LIMIT 10;
"""

# Execute the query and load the result into a DataFrame
top_origins_df = sql_to_df(query_top_origins)

# Display the DataFrame
top_origins_df

# Plot: Balkendiagramm Top-10 origin
Horizontales Barplot der Top-10 origins nach Häufigkeit, sortiert absteigend, Wertebeschriftungen hinzufügen.

In [ ]:
# Plot the horizontal bar chart for the top 10 origins
plt.figure(figsize=(12, 6))
sns.barplot(
    x="c", 
    y="origin", 
    data=top_origins_df, 
    palette="viridis"
)

# Add labels and title
plt.xlabel("Frequency", fontsize=12)
plt.ylabel("Origin", fontsize=12)
plt.title("Top 10 Origins by Frequency", fontsize=14)

# Add value annotations to each bar
for index, value in enumerate(top_origins_df["c"]):
    plt.text(value + 0.5, index, str(value), va="center", fontsize=10)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

# Query: Secret-Wiederholungen über Repos und über Layer+Repos
CTE base: secret_fragments sf JOIN layer_secret_fragments lsf USING(fragment_hash) JOIN repo_layers rl USING(layer_id) JOIN repos r ON r.id=rl.repo_id. Aggregiere pro identischem Secret-Text sf.secret: repo_cnt = COUNT(DISTINCT r.id), layer_repo_cnt = COUNT(DISTINCT (rl.layer_id, r.id)). Erzeuge Schwellen-Buckets (>5, >10, >100, >1000) und zähle Anzahl Secrets je Bucket getrennt für repo_cnt und layer_repo_cnt.

In [ ]:
# Define the SQL query to calculate secret repetition thresholds
query_secret_repeats = """
WITH base AS (
    SELECT 
        sf.secret,
        COUNT(DISTINCT r.id) AS repo_cnt,
        COUNT(DISTINCT (rl.layer_id, r.id)) AS layer_repo_cnt
    FROM secret_fragments sf
    JOIN layer_secret_fragments lsf USING (fragment_hash)
    JOIN repo_layers rl USING (layer_id)
    JOIN repos r ON r.id = rl.repo_id
    GROUP BY sf.secret
),
thresholds AS (
    SELECT 
        'repo_cnt' AS category,
        CASE 
            WHEN repo_cnt > 1000 THEN '>1000'
            WHEN repo_cnt > 100 THEN '>100'
            WHEN repo_cnt > 10 THEN '>10'
            WHEN repo_cnt > 5 THEN '>5'
            ELSE NULL
        END AS bucket,
        COUNT(*) AS secret_count
    FROM base
    GROUP BY 1, 2
    UNION ALL
    SELECT 
        'layer_repo_cnt' AS category,
        CASE 
            WHEN layer_repo_cnt > 1000 THEN '>1000'
            WHEN layer_repo_cnt > 100 THEN '>100'
            WHEN layer_repo_cnt > 10 THEN '>10'
            WHEN layer_repo_cnt > 5 THEN '>5'
            ELSE NULL
        END AS bucket,
        COUNT(*) AS secret_count
    FROM base
    GROUP BY 1, 2
)
SELECT 
    category, 
    bucket, 
    COALESCE(secret_count, 0) AS secret_count
FROM thresholds
WHERE bucket IS NOT NULL
ORDER BY category, bucket;
"""

# Execute the query and load the result into a DataFrame
secret_repeats_df = sql_to_df(query_secret_repeats)

# Pivot the DataFrame for easier plotting
pivot_df = secret_repeats_df.pivot(index="bucket", columns="category", values="secret_count").fillna(0)

# Plot the bar chart for secret repetition thresholds
pivot_df.plot(
    kind="bar",
    figsize=(12, 6),
    color=["#1f77b4", "#ff7f0e"],
    edgecolor="black"
)

# Add labels and title
plt.xlabel("Threshold Buckets", fontsize=12)
plt.ylabel("Number of Secrets", fontsize=12)
plt.title("Secret Repetition Thresholds Across Repositories and Layers+Repositories", fontsize=14)

# Add legend and adjust layout
plt.legend(title="Category", fontsize=10, title_fontsize=12)
plt.tight_layout()

# Show the plot
plt.show()

# Plot: Balkendiagramm Schwellen (>5, >10, >100, >1000), gruppiert (Repos vs. Layer+Repos)
Erzeuge gruppiertes Balkendiagramm je Schwellenwert mit zwei Balken: 'Distinct Repos' und 'Distinct Layer+Repos'. Beschriftungen und Legende hinzufügen.

In [ ]:
# Define the SQL query to calculate secret repetition thresholds
query_secret_repeats = """
WITH base AS (
    SELECT 
        sf.secret,
        COUNT(DISTINCT r.id) AS repo_cnt,
        COUNT(DISTINCT (rl.layer_id, r.id)) AS layer_repo_cnt
    FROM secret_fragments sf
    JOIN layer_secret_fragments lsf USING (fragment_hash)
    JOIN repo_layers rl USING (layer_id)
    JOIN repos r ON r.id = rl.repo_id
    GROUP BY sf.secret
),
thresholds AS (
    SELECT 
        'repo_cnt' AS category,
        CASE 
            WHEN repo_cnt > 1000 THEN '>1000'
            WHEN repo_cnt > 100 THEN '>100'
            WHEN repo_cnt > 10 THEN '>10'
            WHEN repo_cnt > 5 THEN '>5'
            ELSE NULL
        END AS bucket,
        COUNT(*) AS secret_count
    FROM base
    GROUP BY 1, 2
    UNION ALL
    SELECT 
        'layer_repo_cnt' AS category,
        CASE 
            WHEN layer_repo_cnt > 1000 THEN '>1000'
            WHEN layer_repo_cnt > 100 THEN '>100'
            WHEN layer_repo_cnt > 10 THEN '>10'
            WHEN layer_repo_cnt > 5 THEN '>5'
            ELSE NULL
        END AS bucket,
        COUNT(*) AS secret_count
    FROM base
    GROUP BY 1, 2
)
SELECT 
    category, 
    bucket, 
    COALESCE(secret_count, 0) AS secret_count
FROM thresholds
WHERE bucket IS NOT NULL
ORDER BY category, bucket;
"""

# Execute the query and load the result into a DataFrame
secret_repeats_df = sql_to_df(query_secret_repeats)

# Pivot the DataFrame for easier plotting
pivot_df = secret_repeats_df.pivot(index="bucket", columns="category", values="secret_count").fillna(0)

# Plot the bar chart for secret repetition thresholds
pivot_df.plot(
    kind="bar",
    figsize=(12, 6),
    color=["#1f77b4", "#ff7f0e"],
    edgecolor="black"
)

# Add labels and title
plt.xlabel("Threshold Buckets", fontsize=12)
plt.ylabel("Number of Secrets", fontsize=12)
plt.title("Secret Repetition Thresholds Across Repositories and Layers+Repositories", fontsize=14)

# Add legend and adjust layout
plt.legend(title="Category", fontsize=10, title_fontsize=12)
plt.tight_layout()

# Show the plot
plt.show()

# Query: Feature-Matrix pro Secret (Join secret_fragments ↔ layer_secret_fragments ↔ layer_data ↔ repo_layers ↔ repos)
Baue Feature-Tabelle auf Secret-Ebene (Gruppierung nach sf.secret): Features: median(sf.file_size) AS file_size, mode(sf.file_type) AS file_type; avg(ld.file_count) AS file_count, avg(ld.max_depth) AS max_depth, avg(ld.uncompressed_size) AS uncompressed_size; aus repos: avg(r.pull_count) AS pull_count, avg(CASE WHEN r.is_automated THEN 1 ELSE 0 END) AS is_automated, avg(CASE WHEN r.is_official THEN 1 ELSE 0 END) AS is_official, avg(r.star_count) AS star_count; Zielvariable: repo_occurrences = COUNT(DISTINCT r.id). Lade in DataFrame.

In [ ]:
query_feature_matrix = """
WITH secret_features AS (
    SELECT 
        sf.secret,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY sf.file_size) AS file_size,
        MODE() WITHIN GROUP (ORDER BY sf.file_type) AS file_type,
        AVG(ld.file_count) AS file_count,
        AVG(ld.max_depth) AS max_depth,
        AVG(ld.uncompressed_size) AS uncompressed_size,
        AVG(r.pull_count) AS pull_count,
        AVG(CASE WHEN r.is_automated THEN 1 ELSE 0 END) AS is_automated,
        AVG(CASE WHEN r.is_official THEN 1 ELSE 0 END) AS is_official,
        AVG(r.star_count) AS star_count,
        COUNT(DISTINCT r.id) AS repo_occurrences
    FROM secret_fragments sf
    JOIN layer_secret_fragments lsf USING (fragment_hash)
    JOIN layer_data ld USING (layer_id)
    JOIN repo_layers rl USING (layer_id)
    JOIN repos r ON r.id = rl.repo_id
    GROUP BY sf.secret
)
SELECT * FROM secret_features;
"""

# Execute the query and load the result into a DataFrame
feature_matrix_df = sql_to_df(query_feature_matrix)

# Display the first few rows of the DataFrame
feature_matrix_df.head()

# Feature Engineering: Zielvariable und Encoding
Erzeuge y = log1p(repo_occurrences). One-Hot-Encoding für file_type (Top-k Kategorien, Rest als 'other'), skaliere numerische Features optional. Entferne Zeilen mit fehlenden Kernwerten.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np

# Define the target variable y as log1p(repo_occurrences)
feature_matrix_df["y"] = np.log1p(feature_matrix_df["repo_occurrences"])

# One-Hot-Encoding for file_type (Top-k categories, rest as 'other')
top_k = 10
top_categories = feature_matrix_df["file_type"].value_counts().nlargest(top_k).index
feature_matrix_df["file_type"] = feature_matrix_df["file_type"].apply(
    lambda x: x if x in top_categories else "other"
)

# Define numerical features to scale
numerical_features = [
    "file_size",
    "file_count",
    "max_depth",
    "uncompressed_size",
    "pull_count",
    "star_count",
]

# Define categorical features for encoding
categorical_features = ["file_type"]

# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]), numerical_features),
        ("cat", Pipeline([("imputer", SimpleImputer(strategy="constant", fill_value="missing")), ("onehot", OneHotEncoder(drop="first"))]), categorical_features),
    ]
)

# Apply preprocessing to the feature matrix
X = preprocessor.fit_transform(feature_matrix_df)

# Remove rows with missing core values
feature_matrix_df.dropna(subset=["repo_occurrences"], inplace=True)

# Korrelationen (Pearson und Spearman) berechnen und rangieren
Berechne für alle Features die Pearson- und Spearman-Korrelation mit y. Erzeuge DataFrame mit Feature, pearson_r, spearman_r, abs_pearson, abs_spearman und sortiere nach abs_spearman (oder kombiniertem Score).

In [ ]:
import scipy.stats as stats

# Calculate Pearson and Spearman correlations for all features with the target variable y
correlation_results = []
for feature in numerical_features + list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)):
    if feature in feature_matrix_df.columns:
        pearson_r, _ = stats.pearsonr(feature_matrix_df[feature], feature_matrix_df["y"])
        spearman_r, _ = stats.spearmanr(feature_matrix_df[feature], feature_matrix_df["y"])
        correlation_results.append({
            "feature": feature,
            "pearson_r": pearson_r,
            "spearman_r": spearman_r,
            "abs_pearson": abs(pearson_r),
            "abs_spearman": abs(spearman_r)
        })

# Convert the results to a DataFrame
correlation_df = pd.DataFrame(correlation_results)

# Sort the DataFrame by absolute Spearman correlation (or combined score)
correlation_df.sort_values(by="abs_spearman", ascending=False, inplace=True)

# Display the top correlations
correlation_df.head()

# Plot the top features by absolute Spearman correlation
plt.figure(figsize=(12, 6))
sns.barplot(
    x="abs_spearman",
    y="feature",
    data=correlation_df.head(10),
    palette="coolwarm"
)

# Add labels and title
plt.xlabel("Absolute Spearman Correlation", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.title("Top 10 Features by Absolute Spearman Correlation with Target (y)", fontsize=14)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

# Plot: Balkendiagramm der absoluten Korrelationskoeffizienten
Barplot der Top-Features nach |Spearman| (optional nebeneinander mit |Pearson|). Achsenbeschriftungen und Legende hinzufügen.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the top features by absolute Spearman correlation alongside Pearson correlation
top_features = correlation_df.head(10)

plt.figure(figsize=(14, 8))
bar_width = 0.4
x = range(len(top_features))

# Plot Spearman correlations
plt.barh(
    y=[i - bar_width / 2 for i in x],
    width=top_features["abs_spearman"],
    height=bar_width,
    color="skyblue",
    label="|Spearman|"
)

# Plot Pearson correlations
plt.barh(
    y=[i + bar_width / 2 for i in x],
    width=top_features["abs_pearson"],
    height=bar_width,
    color="salmon",
    label="|Pearson|"
)

# Add feature names as y-axis labels
plt.yticks(x, top_features["feature"], fontsize=10)

# Add labels and title
plt.xlabel("Absolute Correlation", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.title("Top 10 Features by Absolute Correlation with Target (y)", fontsize=14)

# Add legend and adjust layout
plt.legend(title="Correlation Type", fontsize=10, title_fontsize=12)
plt.tight_layout()

# Show the plot
plt.show()

# Optional: Ergebnisse/Abbildungen speichern
Speichere Plots als PNG/SVG und exportiere aggregierte Tabellen (Zeitreihe, Top-10, Schwellen, Korrelationen) als CSV in ein Ausgabe-Verzeichnis.

In [ ]:
import os

# Define the output directory
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Save Zeitreihe plot as PNG and SVG
zeitreihe_plot_path_png = os.path.join(output_dir, "zeitreihe_repos_with_secrets.png")
zeitreihe_plot_path_svg = os.path.join(output_dir, "zeitreihe_repos_with_secrets.svg")
plt.figure(figsize=(12, 6))
plt.plot(zeitreihe_df["mon"], zeitreihe_df["repo_count"], marker="o", linestyle="-", color="b", label="Repos with ≥1 Secret")
plt.xlabel("Date", fontsize=12)
plt.ylabel("Number of Repositories", fontsize=12)
plt.title("Repositories with ≥1 Secret per Month (2013–2025)", fontsize=14)
plt.xlim(pd.Timestamp(START_DATE), pd.Timestamp(END_DATE))
plt.xticks(rotation=45)
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.savefig(zeitreihe_plot_path_png)
plt.savefig(zeitreihe_plot_path_svg)
plt.close()

# Save Top 10 Origins plot as PNG and SVG
top_origins_plot_path_png = os.path.join(output_dir, "top_10_origins.png")
top_origins_plot_path_svg = os.path.join(output_dir, "top_10_origins.svg")
plt.figure(figsize=(12, 6))
sns.barplot(x="c", y="origin", data=top_origins_df, palette="viridis")
plt.xlabel("Frequency", fontsize=12)
plt.ylabel("Origin", fontsize=12)
plt.title("Top 10 Origins by Frequency", fontsize=14)
for index, value in enumerate(top_origins_df["c"]):
    plt.text(value + 0.5, index, str(value), va="center", fontsize=10)
plt.tight_layout()
plt.savefig(top_origins_plot_path_png)
plt.savefig(top_origins_plot_path_svg)
plt.close()

# Save Secret Repetition Thresholds plot as PNG and SVG
secret_repeats_plot_path_png = os.path.join(output_dir, "secret_repetition_thresholds.png")
secret_repeats_plot_path_svg = os.path.join(output_dir, "secret_repetition_thresholds.svg")
pivot_df.plot(kind="bar", figsize=(12, 6), color=["#1f77b4", "#ff7f0e"], edgecolor="black")
plt.xlabel("Threshold Buckets", fontsize=12)
plt.ylabel("Number of Secrets", fontsize=12)
plt.title("Secret Repetition Thresholds Across Repositories and Layers+Repositories", fontsize=14)
plt.legend(title="Category", fontsize=10, title_fontsize=12)
plt.tight_layout()
plt.savefig(secret_repeats_plot_path_png)
plt.savefig(secret_repeats_plot_path_svg)
plt.close()

# Save Correlation plots as PNG and SVG
correlation_plot_path_png = os.path.join(output_dir, "top_features_correlation.png")
correlation_plot_path_svg = os.path.join(output_dir, "top_features_correlation.svg")
plt.figure(figsize=(14, 8))
bar_width = 0.4
x = range(len(top_features))
plt.barh(y=[i - bar_width / 2 for i in x], width=top_features["abs_spearman"], height=bar_width, color="skyblue", label="|Spearman|")
plt.barh(y=[i + bar_width / 2 for i in x], width=top_features["abs_pearson"], height=bar_width, color="salmon", label="|Pearson|")
plt.yticks(x, top_features["feature"], fontsize=10)
plt.xlabel("Absolute Correlation", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.title("Top 10 Features by Absolute Correlation with Target (y)", fontsize=14)
plt.legend(title="Correlation Type", fontsize=10, title_fontsize=12)
plt.tight_layout()
plt.savefig(correlation_plot_path_png)
plt.savefig(correlation_plot_path_svg)
plt.close()

# Save DataFrames as CSV
zeitreihe_csv_path = os.path.join(output_dir, "zeitreihe_repos_with_secrets.csv")
top_origins_csv_path = os.path.join(output_dir, "top_10_origins.csv")
secret_repeats_csv_path = os.path.join(output_dir, "secret_repetition_thresholds.csv")
correlation_csv_path = os.path.join(output_dir, "top_features_correlation.csv")

zeitreihe_df.to_csv(zeitreihe_csv_path, index=False)
top_origins_df.to_csv(top_origins_csv_path, index=False)
secret_repeats_df.to_csv(secret_repeats_csv_path, index=False)
correlation_df.to_csv(correlation_csv_path, index=False)